Check GPU

In [1]:
!nvidia-smi

Wed Apr 27 18:54:21 2022       
+-----------------------------------------------------------------------------+
| NVIDIA-SMI 460.32.03    Driver Version: 460.32.03    CUDA Version: 11.2     |
|-------------------------------+----------------------+----------------------+
| GPU  Name        Persistence-M| Bus-Id        Disp.A | Volatile Uncorr. ECC |
| Fan  Temp  Perf  Pwr:Usage/Cap|         Memory-Usage | GPU-Util  Compute M. |
|                               |                      |               MIG M. |
|===============================+======================+======================|
|   0  Tesla P100-PCIE...  Off  | 00000000:00:04.0 Off |                    0 |
| N/A   34C    P0    28W / 250W |      0MiB / 16280MiB |      0%      Default |
|                               |                      |                  N/A |
+-------------------------------+----------------------+----------------------+
                                                                               
+-------

Load Google Drive 

In [2]:
from google.colab import drive #Only if you are using Google Drive
drive.mount('/content/gdrive')
drive.mount("/content/gdrive", force_remount=True)
%cd /content/gdrive/My\ Drive/

Mounted at /content/gdrive
Mounted at /content/gdrive
/content/gdrive/My Drive


Location of spectograms for home location, small sample testing

In [3]:
cd loc_home/

/content/gdrive/My Drive/loc_home


In [4]:
%tensorflow_version  1.x

TensorFlow 1.x selected.


**Model**

In [5]:
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
from glob import glob

training_dir = './dataset/train/' #Define the directory of each
validation_dir = './dataset/validation/'
test_dir = './dataset/test'

folders = glob(training_dir + '/*')
num_classes = len(folders)
print ('Total Classes = ' + str(num_classes))

Total Classes = 52


Import Libraries

In [6]:
# importing requried libraries
from keras import applications
from keras.models import Model
import keras 
from keras.applications.vgg16 import VGG16 #Importing the VGG16 Model
from keras.layers import Flatten, Dense, Dropout, GlobalAveragePooling2D,BatchNormalization
from keras.applications import VGG16, MobileNetV2
from keras.optimizers import SGD
import tensorflow as tf
from keras.preprocessing.image import ImageDataGenerator
from keras.applications.vgg16 import preprocess_input  

Using TensorFlow backend.


**Data pre-processing**

In [7]:
IMAGE_SIZE = [224,224]   

# Use ImageDataGenerator to import data with labels easily into the model.
# We will use function to rescale, rotate, zoom, flip etc
# This class doesn’t affect the data stored on the disk. 
# This class alters the data on the go while passing it to the model.

# we use the preprocess_input function from VGG16 to normalize the input data
#training_datagen = image_dataset_from_directory(training_dir, labels='inferred', color_mode='rgb',image_size=(224,224))
#training_datagen = ImageDataGenerator(preprocessing_function=preprocess_input)
training_datagen = ImageDataGenerator(rescale=1./255, zoom_range=0.3, rotation_range=40, 
                                      width_shift_range=0.3, height_shift_range=0.3, 
                                      shear_range=0.3, horizontal_flip=True, fill_mode="nearest",
                                      preprocessing_function=preprocess_input)

#validation_datagen = ImageDataGenerator(rescale = 1./255, preprocessing_function=preprocess_input)
validation_datagen = ImageDataGenerator(preprocessing_function=preprocess_input)

training_generator = training_datagen.flow_from_directory(training_dir, target_size = IMAGE_SIZE, batch_size = 255, class_mode = 'categorical') #In this attempt, I am increasing the batch size to 255 in both training and validation
validation_generator = validation_datagen.flow_from_directory(validation_dir, target_size = IMAGE_SIZE, batch_size = 255, class_mode = 'categorical')
training_generator.class_indices


Found 4055 images belonging to 52 classes.
Found 1947 images belonging to 69 classes.


{'Alarm': 0,
 'Applause': 1,
 'Bathtub_filling_or_washing': 2,
 'Bell': 3,
 'Boiling': 4,
 'Chatter': 5,
 'Chink_and_clink': 6,
 'Chirp_and_tweet': 7,
 'Coin_dropping': 8,
 'Computer_keyboard': 9,
 'Cupboard_open_or_close': 10,
 'Cutlery_and_silverware': 11,
 'Dishes_and_pots_and_pans': 12,
 'Domestic_sounds_and_home_sounds': 13,
 'Door': 14,
 'Doorbell': 15,
 'Drawer_open_or_close': 16,
 'Drip': 17,
 'Engine': 18,
 'Fart': 19,
 'Fill_with_liquid': 20,
 'Frying_food': 21,
 'Idling': 22,
 'Keys_jangling': 23,
 'Knock': 24,
 'Male_speech_and_man_speaking': 25,
 'Meow': 26,
 'Microwave_oven': 27,
 'Packing_tape_and_duct_tape': 28,
 'Raindrop': 29,
 'Rattle': 30,
 'Run': 31,
 'Scissors': 32,
 'Screech': 33,
 'Sink_filling_or_washing': 34,
 'Slam': 35,
 'Sliding_door': 36,
 'Squeak': 37,
 'Subway_and_metro_and_underground': 38,
 'Tap': 39,
 'Tearing': 40,
 'Telephone': 41,
 'Toilet_flush': 42,
 'Tools': 43,
 'Trickle_and_dribble': 44,
 'Typewriter': 45,
 'Typing': 46,
 'Walk_and_footsteps':

In [8]:

#Loading VGG16 Model
IMAGE_SIZE = [224,224]   

#we use the Include_top=False to remove the classification 
#layer that was trained on the ImageNet dataset and set the model as not trainable
vgg_model = VGG16(input_shape = IMAGE_SIZE + [3], weights = 'imagenet', include_top = False)  

for layer in vgg_model.layers:
      layer.trainable = False # Non trainable weights

# Create Dense Layers
# Add the last layers (Flatten and Dense layers) for our problem
x = vgg_model.output
x = Flatten()(x) 
# After creating all the convolution, pass the data to the dense layer to flatten the vector 
# which comes out of the convolutions (Refer to VGG16 image dimention layers above)
x = Dense(num_classes, activation = 'softmax')(x)
x = Dense(69, activation = 'softmax')(x) 
#x = Dense(69, activation = 'softmax')(x) 
x = Dropout(0.5)(x) # Dropout layer to reduce overfitting
transfer_model = Model(inputs = vgg_model.input, outputs = x)
# Compile model, for this we will be using ADAM optimiser to reach to the global minima while training our model
learning_rate= 5e-5
transfer_model.compile(loss="categorical_crossentropy", optimizer='adam', metrics=['accuracy'])
#history = transfer_model.fit(X_train, y_train, batch_size = 1, epochs=50, validation_data=(X_test,y_test))

#transfer_model.compile(loss='categorical_crossentropy', optimizer='adam', metrics=['accuracy']) 
transfer_model.summary() #check summary of the model using this command


Instructions for updating:
If using Keras pass *_constraint arguments to layers.

58892288/58889256 [==============================] - 3s 0us/step
Model: "model_1"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
input_1 (InputLayer)         (None, 224, 224, 3)       0         
_________________________________________________________________
block1_conv1 (Conv2D)        (None, 224, 224, 64)      1792      
_________________________________________________________________
block1_conv2 (Conv2D)        (None, 224, 224, 64)      36928     
_________________________________________________________________
block1_pool (MaxPooling2D)   (None, 112, 112, 64)      0         
_________________________________________________________________
block2_conv1 (Conv2D)        (None, 112, 112, 128)     73856     
_________________________________________________________________
block2_conv2 (Conv2D)        (None, 112, 112

In [9]:
history = transfer_model.fit_generator(training_generator,
                   steps_per_epoch = 16, #Number of iterations = number of training images (4055) / batch size (255)  
                   epochs = 10, 
                   validation_data = validation_generator,
                   validation_steps = 2, #same for validation data 300 validation images
                   shuffle = True) 
model.save('AED_A_100_vgg16_home_255_Param.h5')

Instructions for updating:
Use tf.where in 2.0, which has the same broadcast rule as np.where

Epoch 1/10


/tensorflow-1.15.2/python3.7/keras/utils/data_utils.py:616: UserWarning: The input 7 could not be retrieved. It could be because a worker has died.
  UserWarning)


ValueError: ignored

In [ ]:
from keras.models import load_model
model = load_model('A_100_vgg16_v10e_255_Param.h5')

In [ ]:
!pip install 'h5py==2.10.0' --force-reinstall #Sometimes the above makes error, use this command only if the above made an error

Model Testing

In [ ]:
from keras.preprocessing.image import array_to_img, img_to_array, load_img
from keras.applications.vgg16 import preprocess_input
import numpy as np
import os
img_size = 224
clssN = ['Heidelberglaan15','Padualaan101','Padualaan97']
count = 0
countT = 0
for bl in clssN:
  src = './dataset2/Test/' + bl 
  imgs = os.listdir(src)
  count = 0
  countT = len(imgs) #total number of images per class
  for img in imgs: 
    im = load_img(src + "/" + img)
    w,h = im.size
    im = im.resize((int(w*0.2),int(h*0.2)))
    im = im.resize((img_size,img_size))
    x = img_to_array(im)  
    x = np.expand_dims(x, axis=0)
    x = preprocess_input(x)
    x/=255.
    probs = model.predict(x, verbose=0)
    maxDet = max(probs[0])
    maxIdx = list(probs[0]).index(maxDet) #predicted class with maximum probability
    if (clssN[maxIdx]==bl): 
      count = count + 1  #count for true predictions (true positives)
    else:
      print(f'Wrong prediction of {bl}  {img}. Predicted as {clssN[maxIdx]}') 
  print(f'test accuracy for {bl} is {str(count/countT)}')

Using the below code, you will see the output of what building class the model has predicted for this specific image.

In [ ]:
from keras.preprocessing.image import array_to_img, img_to_array, load_img
from keras.applications.vgg16 import preprocess_input
import numpy as np
import os
img_size = 224
clssN = ['Heidelberglaan15','Padualaan101','Padualaan97']
imgDir = "dataset2/Test/Padualaan101/20211220_121005.jpg"
im = load_img(imgDir)
w,h = im.size
im = im.resize((int(w*0.2),int(h*0.2)))
im = im.resize((img_size,img_size))
x = img_to_array(im)  
x = np.expand_dims(x, axis=0)
x = preprocess_input(x)
x/=255.
probs = model.predict(x, verbose=0)
maxDet = max(probs[0])
maxIdx = list(probs[0]).index(maxDet) #predicted class with maximum probability
print(clssN[maxIdx])

Plotting loss and accuracy

In [ ]:
from matplotlib import pyplot
# plot loss during training
pyplot.subplot(211)
pyplot.title('Loss')
pyplot.plot(history.history['loss'], label='train')
pyplot.plot(history.history['val_loss'], label='test')
pyplot.legend()
# plot accuracy during training
pyplot.subplot(212)
pyplot.title('Accuracy')
pyplot.plot(history.history['accuracy'], label='train')
pyplot.plot(history.history['val_accuracy'], label='test')
pyplot.legend()
pyplot.show()